# Mode locations for ARF basic

In [ ]:
from fiberamp.fiber.microstruct.pbg import ARF2
from ngsolve.webgui import Draw
import ngsolve as ng

In [ ]:
A = ARF2(poly_core=True, name='basic', curve=3, refine=0, e=.002)


In [ ]:
Draw(A.mesh)

# Fundamental Mode

In [ ]:
center = 3.345
radius = .1     
alpha = 5


betas_fund, _, Es_fund, phis_fund, _ = A.leakyvecmodes(ctr=center, rad=radius, alpha=alpha,
                                          nspan=4, npts=4, p=2, niterations=10,
#                                           rhoinv=.99, quadrule='ellipse_trapez_shift',
                                          nrestarts=0, stop_tol=1e-9)

In [ ]:
for e in Es_fund:
    Draw(e.real, A.mesh, vectors={'grid_size':200})

In [ ]:
for phi in phis_fund:
    Draw(phi, A.mesh)

# Higher Order Modes

ctr=6.25 r = .05



In [ ]:
center = 6.5
radius = 2   
alpha = 5


betas_hom, _, Es_hom, phis_hom, _ = A.leakyvecmodes(ctr=center, rad=radius, alpha=alpha,
                                          nspan=10, npts=6, p=1, niterations=10,
                                          rhoinv=.99, quadrule='ellipse_trapez_shift',
                                          nrestarts=0, stop_tol=1e-9)

In [ ]:
for e in Es_hom:
    Draw(e.real, A.mesh, vectors={'grid_size':200})

# Refined search

### Transverse Electric:

In [ ]:
center_te =12.744
radius = .005   
alpha = 5


betas_te, _, Es_te, phis_te, _ = A.leakyvecmodes(ctr=center_te, rad=radius, alpha=alpha,
                                          nspan=4, npts=4, p=2, niterations=10,
                                          nrestarts=0, stop_tol=1e-9)

In [ ]:
for e in Es_te:
    Draw(e.real, A.mesh, vectors={'grid_size':200})

### HOM

In [ ]:
center_te =12.766
radius = .005   
alpha = 5


betas_hom, _, Es_hom, phis_hom, _ = A.leakyvecmodes(ctr=center_te, rad=radius, alpha=alpha,
                                          nspan=4, npts=4, p=2, niterations=10,
                                          nrestarts=0, stop_tol=1e-9)

In [ ]:
for e in Es_hom:
    Draw(e.real, A.mesh, vectors={'grid_size':200})

# Orthogonality Check



In [ ]:
# Set choice for fundamental mode
Etv_fund, phi_fund, beta_fund = Es_fund[0], phis_fund[0], betas_fund[0]

# Get H field for fundamental mode
Htv_fund, Hz_fund = A.HfromE(Etv_fund, phi_fund, beta_fund)

# Find current modal power
E_fund_power = A.power(Etv_fund, Htv_fund)
E_fund_power

In [ ]:
# Set choice for higher order mode (here Transverse Electric type)
Etv_TE, Ephi_TE, beta_TE = Es_hom[1], phis_hom[1], betas_hom[1]

# Get H field for hom
Htv_TE, Hz_TE = A.HfromE(Etv_TE, Ephi_TE, beta_TE)

# Find current modal power
E_hom_power = A.power(Etv_TE, Htv_TE)

E_hom_power

In [ ]:
# Check orthogonality (for power (maxwell) inner product)
A.power(Etv_fund, Htv_TE)

In [ ]:
# Create full E field for fundamental
Ez_fund = -1j/beta_fund * phi_fund
E_fund_full = ng.CoefficientFunction((Etv_fund[0], Etv_fund[1], Ez_fund))

# Create full H field for fundamental
H_fund_full = ng.CoefficientFunction((Htv_fund[0], Htv_fund[1], Hz_fund))

# Create full E field for higher order mode
Ez_TE = -1j/beta_TE * Ephi_TE
E_TE_full = ng.CoefficientFunction((Etv_TE[0], Etv_TE[1], Ez_TE))

# Create full hom H field
H_TE_full = ng.CoefficientFunction((Htv_TE[0], Htv_TE[1], Hz_TE))





# Full Field dot product Orthogonality

In [ ]:
t1 = ng.InnerProduct(E_fund_full, E_TE_full)
t2 = ng.InnerProduct(H_fund_full, H_TE_full)
ng.Integrate(t1 + t2, A.mesh)

# Component-wise Orthogonality

In [ ]:
# E field componentwise 

for i in range(3):
    print(ng.Integrate(ng.InnerProduct(E_fund_full[i], E_TE_full[i]), A.mesh))

In [ ]:
# H field componentwise 

for i in range(3):
    print(ng.Integrate(ng.InnerProduct(H_fund_full[i], H_TE_full[i]), A.mesh))

In [ ]:
# E field componentwise 

for i in range(3):
    print(ng.Integrate(ng.InnerProduct(E_fund_full[i], H_TE_full[i]), A.mesh))